# Important:
Before publishing, go back and add in filter for artworks that are public domain. Not sure if I missed this earlier or if it is all good to go.

In [2]:
import numpy as np
import pandas as pd
import dash
from dash import dash_table
from dash import dcc
import plotly.figure_factory as ff
from dash import html
from dash.dependencies import Input, Output
from sqlalchemy import create_engine
import psycopg2
import os

POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
import dash_bootstrap_components as dbc

from bson.json_util import loads, dumps

In [3]:
import markdownify as mdf

In [4]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'artipy'
))

In [5]:
project_description = '''
Synesthesia is a blending of the senses, a neurological phenomenon where stimuli in one sensory pathway can trigger 
involuntary responses in another. The goal of this project is to develop an interactive platform that explores the 
intersection of art, art history, and music, aiming to heighten engagement with the arts and enrich the user's experience.

Art is a means through which one can learn about history, culture, emotion, humanity and more and yet, outside of museums 
and schools, people are not exposed to much art. Even in museums, the time people spend viewing 
art is quite short. In contrast to this, music has become much more ingrained in our daily lives, 
especially with the rise of music streaming platforms. With this in mind, I ventured to explore the idea of pairing music 
with art in the form of an interactive dashboard.  
'''

the_data = '''
This dashboard uses data from the Spotify Web API and the Art Institute of Chicago API. The Spotify Web API 
provides access to information on songs such as artist, year, and title as well as interpretive Audio Features which were 
used in the enhance feature of this dashboard. The Art Institute of Chicago API is a detailed database of the museum’s public 
data with over two-hundred thousand works and their corresponding information available.
'''

how_it_works = '''
There are two main components to each tab of this dashboard: the Music Box and the Art Gallery.

The music box allows users to switch between songs from popular musicians who are known to be connected 
to the art movement in focus and play a 30s preview clip of each song.
The enhance toggle in the music box switches the playlist to display songs which have similar Spotify Audio Features to the 
songs of the selected artists. This allows for an exploration into songs which vary in genre, language, time-period, and more
but still suit the stylistic theme of the artistic era in question.

The art gallery allows users to flip through selected works under the chosen genre, provided by the Art Institute of Chicago. 
'''

In [6]:
impressionism_blurb = '''
Impressionism  emerged in France during the late 1800's. Rather than focusing on detailed realism, Impressionist artists 
sought to convey the atmosphere, mood, and sensory experience of a scene by including visible brushstrokes, a 
focusing on the play of light, and emphasizing the overall visual experience rather than creating a detailed representation. 


Impressionistic elements in music include static harmony, emphasis on instrumental timbres that create a shimmering 
interplay of “colors”, melodies that lack directed motion, surface ornamentation that obscures or substitutes for melody, 
and an avoidance of traditional musical form. Composers most often associated with Impressionism include Claude Debussy and 
Maurice Ravel. 
'''

popart_blurb = '''
By creating paintings or sculptures of mass culture objects and media stars, the Pop Art movement aimed to blur the boundaries between "high" art and "low" culture. The concept that there is no hierarchy of culture and that art may borrow from any source has been one of the most influential characteristics of Pop Art. Pop artists recognized that there is no unmediated access to anything, be it the soul, the natural world, or the built environment. Pop artists believed everything is inter-connected, and therefore sought to make those connections literal in their artwork.

Some of the most famous Pop artists began their careers in commercial art: Andy Warhol was a highly successful magazine illustrator and graphic designer; Ed Ruscha was also a graphic designer, and James Rosenquist started his career as a billboard painter. Their background in the commercial art world trained them in the visual vocabulary of mass culture as well as the techniques to seamlessly merge the realms of high art and popular culture.

Andy Warhol and his work influenced pop culture in general, especially music. Warhol worked on creating albums for artists such as The Rolling Stones and Aretha Franklin, Diana Ross and John Lennon.
His best-known cover is from the band Velvet Underground and what many may not know is that Andy Warhol was also the band’s manager during this period.

“Everything is beautiful. Pop is everything.” — Andy Warhol
'''

In [7]:
external_stylesheets = [dbc.themes.LUX]

In [8]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [ 
        #Stuff on top
        html.Img(src = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Art_Institute_of_Chicago_logo.svg/768px-Art_Institute_of_Chicago_logo.svg.png',
                                    style = {'width': '7%', 'float': 'right'}),
        html.Img(src = 'https://upload.wikimedia.org/wikipedia/commons/thumb/7/74/Spotify_App_Logo.svg/2048px-Spotify_App_Logo.svg.png',
                                    style = {'width': '7%', 'float': 'right'}),
        html.H1("Synesthesia:"),
        html.H2("An Exploration of Various Artistic Movements Through Music"),

        #Tabs

        dbc.Tabs([
                    dbc.Tab(label = 'Home', tab_id='home', children = [
                    #describe the layout and purpose of project
                        html.Div([
                            html.H4("The Project: What is Synesthesia?"),
                            dcc.Markdown(project_description),
                            html.H4("The Data"),
                            dcc.Markdown(the_data)
                        ], style = {'width': '40%', 'float': 'left'}),
                        
                        html.Div([
                            html.H4("Click to learn more about each category:"),
                            #accordian
                            dbc.Accordion(
                                [
                                    dbc.AccordionItem(
                                        [
                                            dcc.Markdown(impressionism_blurb),
                                        ],
                                        title="Impressionism",
                                    ),
                                    dbc.AccordionItem(
                                        [
                                            dcc.Markdown(popart_blurb),
                                        ],
                                        title="Pop Art",
                                    ),
                                    dbc.AccordionItem(
                                        "This is the content of the third section",
                                        title="TBD 1",
                                    ),
                                    dbc.AccordionItem(
                                        "This is the content of the third section",
                                        title="TBD 2",
                                    ),
                                ],
                                start_collapsed=True,
                            ),

                            #how it works
                            html.H4(" "),
                            html.H4("Navigating the Dashboard:"),
                            dcc.Markdown(how_it_works),
                        ], style = {'width': '57%', 'float': 'right'})
                    
                    ]),
                    dbc.Tab(label = 'Impressionism', tab_id='impressionism', children = [
                    # stuff for impressionism tab goes here
                        #Music box
                        html.Div([
                            #dcc.Markdown(impressionism_music)
                            html.Div(
                                [
                                    html.Div("Change Songs:"),
                                    dbc.Pagination(id="imp_pagination", max_value=50, fully_expanded=False),
                                    dbc.Label("Toggle to add songs with similar audio features:"),
                                    dbc.Checklist(
                                        options=[
                                            {"label": "Enhance", "value": 1}
                                        ],
                                        value=[0],
                                        id="imp_switches-input",
                                        switch=True,
                                    ),
                                    html.Img(id='imp_album_image'),
                                    dcc.Markdown(id='imp_song_info'),
                                    html.Audio(id = 'imp_preview_url',
                                       controls=True,  # Add audio controls (play, pause, volume, etc.)
                                       style={'width': '100%'})
                                ]
                            )
                        ], style = {'width': '30%', 'float': 'left'}),
                
                        #Art Gallery
                        html.Div([
                                dbc.Pagination(id="art_pagination", max_value=150, fully_expanded=False),
                                html.Img(id='imp_artwork_image', style={'max_height':'500px','width':'60%', 'float':'right'}),
                                dcc.Markdown(id="imp_title"),
                                dcc.Markdown(id="imp_artwork_info"),

                        ], style = {'width': '68%', 'float': 'right', "overflow": "scroll"}) #want to make this not scroll the other half eventually
                    
                ]),
                dbc.Tab(label = 'Pop Art', tab_id='popart', children = [
                    #stuff for pop art tab goes here
                    #Music box
                        html.Div([
                            #dcc.Markdown(impressionism_music)
                            html.Div(
                                [
                                    html.Div("Change Songs:"),
                                    dbc.Pagination(id="pop_pagination", max_value=50, fully_expanded=False),
                                    dbc.Label("Toggle to add songs with similar audio features:"),
                                    dbc.Checklist(
                                        options=[
                                            {"label": "Enhance", "value": 1}
                                        ],
                                        value=[0],
                                        id="pop_switches-input",
                                        switch=True,
                                    ),
                                    html.Img(id='pop_album_image'),
                                    dcc.Markdown(id='pop_song_info'),
                                    html.Audio(id = 'pop_preview_url',
                                       controls=True,  # Add audio controls (play, pause, volume, etc.)
                                       style={'width': '100%'})
                                ]
                            )
                        ], style = {'width': '30%', 'float': 'left'}),
                
                        html.Div([
                                dbc.Pagination(id="popart_pagination", max_value=50, fully_expanded=False),
                                html.Img(id='pop_artwork_image', style={'max_height':'500px','width':'60%', 'float':'right'}),
                                dcc.Markdown(id="pop_title"),
                                dcc.Markdown(id="pop_artwork_info"),

                        ], style = {'width': '68%', 'float': 'right', "overflow": "scroll"}) #want to make this not scroll the other half eventually
    
                ]),
                dbc.Tab(label = 'TBD', tab_id='tbd1', children = [
                    #stuff for TBD tab goes here
                    
                ]),
                dbc.Tab(label = 'TBD', tab_id='tbd2', children = [
                    #stuff for TBD tab goes here
                    
                ])
        ])
    ]
)

@app.callback([Output('pop_album_image', 'src'),
              Output('pop_song_info', 'children'),
              Output('pop_preview_url', 'src')],
             [Input('pop_pagination', 'active_page'),
             Input("pop_switches-input", "value")])

def popart_tab(page, switches_value):

    if not page:
        page = 1

    id0 = '1nJvji2KIlWSseXRSlNYsC'
    id1 = '3WrFJ7ztbogyGnTHbHJFl2'
    id2 = '22bE4uQ6baNwSHPVcDxLCe'
    id3 = '3oDbviiivRWhXwIE8hxkVV'

    myquery = f'''
    SELECT name, artists_names, album_image, preview_url,
           danceability,
           energy,
           speechiness,
           acousticness,
           valence
                   
    FROM spotify
    WHERE first_artist_id = '{id1}'
        OR first_artist_id = '{id2}'
        OR first_artist_id = '{id3}'
        OR first_artist_id = '{id0}'
        AND preview_url != 'None'
    ORDER BY artists_names DESC
    ''' 
    df = pd.read_sql_query(myquery, con=engine)
        
    if len(switches_value) == 2:
        myquery = f'''
        SELECT name,
               preview_url,
               artists_names, 
               album_image
                   
        FROM spotify
        WHERE danceability > {max(df['danceability'].mean().round(3) - .1, 0)} AND danceability < {min(df['danceability'].mean().round(3) + .1, 1)}
            AND energy > {max(df['energy'].mean().round(3) - .1, 0)} AND energy < {min(df['energy'].mean().round(3) + .1, 1)}
            AND speechiness > {max(df['speechiness'].mean().round(3) - .1, 0)} AND speechiness < {min(df['speechiness'].mean().round(3) + .1, 1)}
            AND acousticness > {max(df['acousticness'].mean().round(3) - .1, 0)} AND acousticness < {min(df['acousticness'].mean().round(3) + .1, 1)}
            AND valence > {max(df['valence'].mean().round(3) - .1, 0)} AND valence < {min(df['valence'].mean().round(3) + .1, 1)}
            AND first_artist_id != '{id1}' 
            AND first_artist_id != '{id2}'
            AND first_artist_id != '{id3}'
            AND first_artist_id != '{id0}'
            AND preview_url != 'None'
        ''' 
        df = pd.read_sql_query(myquery, con=engine)
        
    mymarkdown = f'''
    ### {df['name'][page -1]}
    ##### {df['artists_names'][page -1]}
    '''
    return df['album_image'][page -1], mymarkdown, df['preview_url'][page- 1]


@app.callback([Output('imp_album_image', 'src'),
              Output('imp_song_info', 'children'),
              Output('imp_preview_url', 'src')],
             [Input('imp_pagination', 'active_page'),
             Input("imp_switches-input", "value")])

def impressionism_tab(page, switches_value):

    if not page:
        page = 1
        
    id1 = '17hR0sYHpx7VYTMRfFUOmY'
    id2 = '1Uff91EOsvd99rtAupatMP'

    myquery = f'''
    SELECT name, artists_names, album_image, preview_url,
           danceability,
           energy,
           speechiness,
           acousticness,
           valence
                   
    FROM spotify
    WHERE first_artist_id = '{id1}'
        OR first_artist_id = '{id2}'
        AND preview_url != 'None'
    ''' 
    imp_df = pd.read_sql_query(myquery, con=engine)
        
    if len(switches_value) == 2:
        myquery = f'''
        SELECT name,
               preview_url,
               artists_names, 
               album_image
                   
        FROM spotify
        WHERE danceability > {max(imp_df['danceability'].mean().round(3) - .1, 0)} AND danceability < {min(imp_df['danceability'].mean().round(3) + .1, 1)}
            AND energy > {max(imp_df['energy'].mean().round(3) - .1, 0)} AND energy < {min(imp_df['energy'].mean().round(3) + .1, 1)}
            AND speechiness > {max(imp_df['speechiness'].mean().round(3) - .1, 0)} AND speechiness < {min(imp_df['speechiness'].mean().round(3) + .1, 1)}
            AND acousticness > {max(imp_df['acousticness'].mean().round(3) - .1, 0)} AND acousticness < {min(imp_df['acousticness'].mean().round(3) + .1, 1)}
            AND valence > {max(imp_df['valence'].mean().round(3) - .1, 0)} AND valence < {min(imp_df['valence'].mean().round(3) + .1, 1)}
            AND first_artist_id != '{id1}' 
            AND first_artist_id != '{id2}' 
            AND preview_url != 'None'
        ''' 
        imp_df = pd.read_sql_query(myquery, con=engine)
        
    imp_mymarkdown = f'''
    ### {imp_df['name'][page -1]}
    ##### {imp_df['artists_names'][page -1]}
    '''
    return imp_df['album_image'][page -1], imp_mymarkdown, imp_df['preview_url'][page- 1]

@app.callback([Output('imp_artwork_image', 'src'),
               Output('imp_title', 'children'),
              Output('imp_artwork_info', 'children')],
    [Input("art_pagination", "active_page")])

def imp_artworks(page):
    if not page:
        page = 1

    myquery = '''
    SELECT title,
        medium_display,
        date_display,
        artist_info_display,
        place_origin,
        description,
        image_url,
        style_title
                   
    FROM artworks
    WHERE style_title = 'Impressionism'
        AND description != 'None'
    ''' 
    df = pd.read_sql_query(myquery, con=engine)

    info = df['artist_info_display'][page-1]
    info = info.replace("\n", "\n###### ")
    title = f'''
   # {df['title'][page-1]}
   #### {df['date_display'][page-1]}

    
   ###### {info}
   ###### {df['medium_display'][page-1]}
   ###### {df['place_origin'][page-1]}
   '''
    
    mymarkdown = f'''
    {mdf.markdownify(df['description'][page-1])}
    '''
    
    return df['image_url'][page - 1], title, mymarkdown

@app.callback([Output('pop_artwork_image', 'src'),
               Output('pop_title', 'children'),
              Output('pop_artwork_info', 'children')],
    [Input("popart_pagination", "active_page")])

def pop_artworks(page):
    if not page:
        page = 1

    myquery = '''
    SELECT title,
        medium_display,
        date_display,
        artist_info_display,
        place_origin,
        description,
        image_url,
        style_title
                   
    FROM artworks
    WHERE style_title = 'Pop Art'
        AND description != 'None'
    ''' 
    df = pd.read_sql_query(myquery, con=engine)

    info = df['artist_info_display'][page-1]
    info = info.replace("\n", "\n###### ")
    title = f'''
   # {df['title'][page-1]}
   #### {df['date_display'][page-1]}

    
   ###### {info}
   ###### {df['medium_display'][page-1]}
   ###### {df['place_origin'][page-1]}
   '''
    
    mymarkdown = f'''
    {mdf.markdownify(df['description'][page-1])}
    '''
    
    return df['image_url'][page - 1], title, mymarkdown

if __name__ == "__main__":
    app.run_server(mode = 'external', host = "0.0.0.0", debug=True)

# EXPERIMENTATION BELOW

In [18]:
id1 = '3WrFJ7ztbogyGnTHbHJFl2'
id2 = '22bE4uQ6baNwSHPVcDxLCe'
id3 = '3oDbviiivRWhXwIE8hxkVV'

myquery = f'''
    SELECT name, artists_names, album_image, preview_url,
           danceability,
           energy,
           speechiness,
           acousticness,
           valence
                   
    FROM spotify
    WHERE first_artist_id = '{id1}'
        OR first_artist_id = '{id2}'
        OR first_artist_id = '{id3}'
        AND preview_url != 'None'

    ''' 
df = pd.read_sql_query(myquery, con=engine)
len(df)

50

In [19]:
myquery = f'''
        SELECT name,
               preview_url,
               artists_names, 
               album_image
                   
        FROM spotify
        WHERE danceability > {max(imp_df['danceability'].mean().round(3) - .1, 0)} AND danceability < {min(imp_df['danceability'].mean().round(3) + .1, 1)}
            AND energy > {max(imp_df['energy'].mean().round(3) - .1, 0)} AND energy < {min(imp_df['energy'].mean().round(3) + .1, 1)}
            AND speechiness > {max(imp_df['speechiness'].mean().round(3) - .1, 0)} AND speechiness < {min(imp_df['speechiness'].mean().round(3) + .1, 1)}
            AND acousticness > {max(imp_df['acousticness'].mean().round(3) - .1, 0)} AND acousticness < {min(imp_df['acousticness'].mean().round(3) + .1, 1)}
            AND valence > {max(imp_df['valence'].mean().round(3) - .1, 0)} AND valence < {min(imp_df['valence'].mean().round(3) + .1, 1)}
            AND first_artist_id != '{id1}' 
            AND first_artist_id != '{id2}'
            AND first_artist_id != '{id3}' 
            AND preview_url != 'None'
        ''' 
df = pd.read_sql_query(myquery, con=engine)
len(df)

577

In [22]:
page = 4
t = df['title'][page-1].strip()
date = df['date_display'][page-1]
info = df['artist_info_display'][page-1]
medium = df['medium_display'][page-1]
place = df['place_origin'][page-1]

title = f'''
# {t}
#### {date}

##### {info}
##### {medium} 
##### {place}
'''
    
mdf.markdownify(df['description'][page-1])

"Approximately six and a half feet square, *Irises* is one of a series of large paintings Claude Monet undertook during World War I experimenting with familiar motifs on an ever-expanding scale. Lacking a discernible horizon or clear sense of depth, the viewer is both on top of and submerged in this encrusted and disorienting surface, suggestive of water, on which various vegetal and floral forms float.\n\n\nTogether with other related canvases, this work remained in the artist’s Giverny studio long after his death in 1926 and was only rediscovered in the 1950s, having suffered localized damage by shrapnel from shells during World War II. In 1956 the Art Institute’s pioneering curator of modern art, Katherine Kuh, bought this painting from Katia Granoff's gallery in Paris. Kuh recognized a formal affinity between Monet’s late experimental painting and the public’s growing interest in large-scale, abstract works like those by Jackson Pollock, whose [*Greyed Rainbow*](https://www.artic.e